In [ ]:
!pip install transformers==4.44.0
!pip install accelerate==1.2.1
!pip install python-dotenv==1.0.1
!pip install requests==2.31.0
!pip install PyYAML==6.0
!pip install Jinja2==3.1.0
!pip install tornado==6.2
!pip install outlines==0.1.12
!pip install datasets


In [ ]:
import os
import requests          # For making API calls
import json              # For parsing and working with JSON data
import datetime          # For handling dates and validating input
from IPython import get_ipython
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import ipywidgets as widgets
from dotenv import load_dotenv
import outlines
import datasets
import torch


In [ ]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def entityIdForFlights(place):
    url = "https://sky-scanner3.p.rapidapi.com/flights/auto-complete"
    
    querystring = {"query":place}
    
    headers = {
    	"x-rapidapi-key": "YOUR_API_KEY",
    	"x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()['data'][0]['presentation']['skyId']

In [ ]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def entityIdForHotels(place):
        url = "https://sky-scanner3.p.rapidapi.com/hotels/auto-complete"
        
        querystring = {"query": place}
        
        headers = {
        	"x-rapidapi-key": "YOUR_API_KEY",
        	"x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        
        return response.json()['data'][0]['entityId']

In [ ]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def search_flights(Origin, destination, start_date, end_date):
    """
    Queries Flight Scaper's roundtrip flight search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/flights/search-roundtrip"
    headers = {
      "x-rapidapi-key": "YOUR_API_KEY",
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {"fromEntityId": Origin,"ToEntityId":destination,"departDate":start_date,"returnDate":end_date,"rooms":"1","adults":"1"}

    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [ ]:
from dotenv import load_dotenv
import outlines

load_dotenv()

def search_hotels(location, check_in, check_out):
    """
    Queries Flight Scaper's hotel search API.
    """
    url = "https://flights-scaper.p.rapidapi.com/hotels/search"
    headers = {
      "x-rapidapi-key": "YOUR_API_KEY",
      "x-rapidapi-host": "sky-scanner3.p.rapidapi.com"
    }
    params = {"entityId":location,"checkin":check_in,"checkout":check_out,"adults":"1"}

    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [ ]:

class LlamaTravelBot:
    
    def __init__(self):
        self.travel_details = {}  # Store travel details
        self.tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
        self.model = AutoModelForCausalLM.from_pretrained("NousResearch/Hermes-3-Llama-3.2-3B")
        self.generator = pipeline(task="text-generation", model=self.model, tokenizer=self.tokenizer, device=0)
        self.chat_history= [{"role": "system","content": " You are an intelligent and conversational travel assistant. The current year is 2025. Your goal is to help the user plan their trip. 1. Have a natural conversation to get the following details from the user: Required: Origin, Destination, Travel Start Date, Travel End Date.Optional: Budget, Hotel Preferences, (e.g. location, amenities), Flight preferences (e.g., airline, class) 2. Dynamically adjust the conversation based on missing details. 3. Do no assume information; only take into account what the user provides. "
            }]
        self.details={}

    def update_travel_details(self):
         """
        Update the travel details dictionary with new information.
         """
         self.travel_details.update(self.details)

    

    def llama_ask(self, prompt):
        """
        Generate a response from the Llama model.
        """
        
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenized_chat = self.tokenizer.apply_chat_template(self.chat_history, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
        response = self.model.generate(tokenized_chat, max_new_tokens=256, temperature=0.7) 
        decoded_response = self.tokenizer.decode(response[0], skip_special_tokens=True)

        # Append the new response to the chat history
        self.chat_history.append({"role": "assistant", "content": decoded_response})

        # Return only the newest response
        return decoded_response

    def handle_user_input(self):
        """
        Process user input and manage the conversation flow.
        """
        print("Bot: Hi! I'm your travel assistant. How can I help you today?")
    
        while True:
            while not self.is_ready_for_api():              
                user_input = input("You: ")
    
                if user_input.lower() in ["exit", "quit"]:                   
                    print("Bot: Goodbye! Safe travels!")
                    return  # Use return instead of break to exit the method
                
                # Add user input to the chat history
                self.chat_history.append({"role":"user","content":f"{user_input}"})
                
                llama_response = self.llama_ask(user_input)
                print(f"Bot: {llama_response}")              
                self.details = self.extract_details_from_input(user_input)
                self.update_travel_details()
    
            if self.is_ready_for_api():
                while True:
                    print("Bot: I have gathered all the necessary details. Would you like to proceed with your request? (Y/N)")
                    self.chat_history.append({"role": "assistant", "content": "I have gathered all the necessary details. Would you like to proceed with your request? (Y/N)"})
                    user_input = input("User: ")
                    
                    if user_input.upper() == "Y":
                        print("Bot: Great! Fetching the available flights and hotels for you...")
                        self.fetch_and_present_results()
                        return  # Exit the method after fetching and presenting results
                    
                    elif user_input.upper() == "N":
                        print("Bot: No problem! Please let me know what additional details you'd like to provide.")
                        self.chat_history.append({"role": "assistant", "content": "No problem! Please let me know what additional details you'd like to provide."})
                        
                        additional_input = input("You: ").strip()  # Strip leading/trailing whitespace
                        self.chat_history.append({"role": "user", "content": additional_input})
                        
                        # Extract additional details and update the travel details
                        self.details = self.extract_details_from_input(additional_input)
                        self.update_travel_details()
                    
                    else:
                        print("Bot: Oops! It seems there was a misunderstanding. Please enter 'Y' to proceed or 'N' to provide more details.")
                        self.chat_history.append({"role": "assistant", "content": "Oops! It seems there was a misunderstanding. Please enter 'Y' to proceed or 'N' to provide more details."})
    

    def extract_details_from_input(self, user_input):
        """
        Use the LLM to analyze user input and extract key details.
        """
        schema = {
            "type": "object",
            "properties": {
                "Origin": {"type": "string", "default": None},
                "destination": {"type": "string", "default": None},
                "Travel_start_date": {"type": "string", "format": "date", "default": None},
                "Travel_end_date": {"type": "string", "format": "date", "default": None},
                "Budget": {"type": "string", "default": None},
                "Hotel Preferences": {"type": "string", "default": None},
                "Flight Preferences": {"type": "string", "default": None}
            },
            "required": ["Origin", "Destination", "Travel start date", "Travel end date"]
        }

        prompt = f"""I want you to analyze and extract the following details from the user's chat history so far: '{self.chat_history}'.
        

        Details to be extracted:
        Required: Origin, Destination, Travel Start Date, Travel End Date (The current year is 2025).
        Optional: Budget, Hotel Preferences, (e.g., location, amenities), Flight preferences (e.g., airline, class)
        
        Provide a JSON object according to the following schema:
        - Origin: (type:string) 
        - destination: (type:string)
        - Travel_start_date: (type:string) (in YYYY-MM-DD format)
        - Travel_end_date: (type:string) (in YYYY-MM-DD format)
        - budget: (type:string)
        - Hotel Preferences: (type:string)
        - Flight Preferences: (type:string)

        If any detail is not provided you **MUST** set it to null.
        """

        model = outlines.models.transformers("NousResearch/Hermes-3-Llama-3.2-3B")
        
        # Convert schema to JSON string
        schema_json = json.dumps(schema)
        
        generator = outlines.generate.json(model, schema_json)  # Use the JSON string
        response = generator(prompt)
        print(response)

        try:
            if isinstance(response, dict):
                response = json.dumps(response)

            # Parse the JSON response from the LLM
            extracted_details = json.loads(response)
            return extracted_details
        except json.JSONDecodeError:
            print("Error parsing the LLM's response.")
            return {}


    def is_ready_for_api(self):
        """
        Check if all required fields are collected and contain valid values.
        """
        required_fields = ["Origin", "destination", "Travel_start_date", "Travel_end_date"]
        invalid_values = {"N/A", "null", "none", ""}  # Set of invalid values (case-insensitive)

        # Ensure all required fields are present and contain valid values
        return all(
            field in self.travel_details
            and str(self.travel_details[field]).strip().lower() not in invalid_values
            for field in required_fields
        )
    def fetch_and_present_results(self):
        """
        Use APIs to fetch results and present them to the user.
        """
        Origin = self.travel_details["Origin"]
        destination = self.travel_details["destination"]
        start_date = self.travel_details["Travel_start_date"]
        end_date = self.travel_details["Travel_end_date"]
        budget = self.travel_details.get("budget", "No budget specified")

        print(Origin)
        print(destination)
        print(start_date)
        print(end_date)
        print(budget)
        
        originid= entityIdForFlights(Origin)
        destinationid1= entityIdForFlights(destination)
        destinationid2= entityIdForHotels(destination)

        print(originid)
        print(destinationid1)
        print(destinationid2)
        print(type(originid))
        print(type(destinationid1))
        print(type(destinationid2))
        
        # Fetch flight and hotel data
        flights = search_flights(originid, destinationid1, start_date, end_date)
        print(flights)

        # Present results
        print("\nFlights:")
           
        top_itineraries = flights['data']['itineraries']
        
        # Iterate over the top 3 itineraries
        for i in range(min(3, len(top_itineraries))):
            itinerary = top_itineraries[i]
            price = itinerary['price']['formatted']
            
            print(f"*** Itinerary {i + 1} ***")
            print(f"Price: {price}")
            
            # Loop through each leg in the itinerary
            for j, leg in enumerate(itinerary['legs']):
                origin = leg['origin']['name']
                destination = leg['destination']['name']
                duration = leg['durationInMinutes']
                departure = leg['departure']
                arrival = leg['arrival']
                
                # Determine whether it's outbound or return
                flight_type = "Outbound flight" if j == 0 else "Return flight"
                
                print(f"{flight_type}:")
                print(f"  Origin airport: {origin}")
                print(f"  Destination airport: {destination}")
                print(f"  Duration: {duration} minutes")
                print(f"  Departure time: {departure}")
                print(f"  Arrival time: {arrival}")
            print("\n")
         
        


      # Sample code to iterate through top 3 hotels and showcase desired details
        hotels = search_hotels(destinationid2, start_date, end_date)
        print(hotels)
        hotel_cards = hotels['data']['results']['hotelCards']
        print('\nHotels:')        
        # Iterate over the top 3 hotels
        for i, hotel in enumerate(hotel_cards[:3]):
            hotel_name = hotel['name']
            review_score = hotel['reviewsSummary']['score']
            location = hotel['relevantPoiDistance']
            cheapest_price = hotel['cheapestPrice']
        
            print(f"Hotel {i + 1}:")
            print(f"  Name: {hotel_name}")
            print(f"  Review Score: {review_score}")
            print(f"  Location : {location}")
            print(f"  Cheapest Price Available: {cheapest_price}")
            print("-" * 30)

# Interactive session
if __name__ == "__main__":
    bot = LlamaTravelBot()
    bot.handle_user_input()